<a href="https://colab.research.google.com/github/ramapu12/trabajoFinal/blob/main/trabajoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

# Cargar los datos
url = 'https://raw.githubusercontent.com/ramapu12/trabajoFinal/59399a459ba091dfd304faa2e1bdd509b56d5477/Life%20Expectancy%20Data.csv'
df = pd.read_csv(url)



# Agrupar por país y calcular la media de esperanza de vida
mean_life_by_country = df.groupby('Country')['Life expectancy '].mean()

# Ordenar de mayor a menor
top_countries = mean_life_by_country.sort_values(ascending=False)

# Mostrar los 10 países con mayor esperanza de vida media
df_sorted = df.sort_values(by='Life expectancy ', ascending=False)

# Mostrar los primeros 10 registros
print(df_sorted[['Country', 'Year', 'Life expectancy ']].head(10))

          Country  Year  Life expectancy 
2513       Sweden  2007              89.0
2433        Spain  2007              89.0
938        France  2007              89.0
915       Finland  2014              89.0
1916       Norway  2009              89.0
1293        Italy  2004              89.0
995       Germany  2014              89.0
937        France  2008              89.0
241       Belgium  2014              89.0
1850  New Zealand  2010              89.0
